<a href="https://colab.research.google.com/github/pratiksrm99/Abstractive_Text_Summerization/blob/main/Abstractive_TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install pywsd
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wn: filename=wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from datasets import load_dataset
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from pywsd.lesk import adapted_lesk, simple_lesk, cosine_lesk
import pandas as pd
import numpy as np
import spacy
import re

Warming up PyWSD (takes ~10 secs)... took 9.318150043487549 secs.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
dataset = load_dataset('gigaword')

Generating train split:   0%|          | 0/3803957 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/189651 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1951 [00:00<?, ? examples/s]

Dataset gigaword downloaded and prepared to /root/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [48]:
df = pd.DataFrame(columns= ["Document", "Summary"])
df["Document"] = dataset.get("test")["document"]
df["Summary"] = dataset.get("test")["summary"]

In [49]:
# df["Split_text"] = df["Document"].apply(str.split)
# df["Article_length"] = df["Split_text"].apply(len)

In [50]:
df.head()

,Document,Summary
0,japan 's nec corp. and UNK computer corp. of t...,nec UNK in computer sales tie-up
1,the sri lankan government on wednesday announc...,sri lanka closes schools as war escalates
2,police arrested five anti-nuclear protesters t...,protesters target french research ship
3,factory orders for manufactured goods rose #.#...,us september factory orders up #.# percent
4,the bank of japan appealed to financial market...,bank of UNK UNK for calm in financial markets


In [51]:
sent = "he is sitting on the bank of the river watching a speedboat."
nlp = spacy.load("en_core_web_sm")

def one_func(x):
  try:
    if x.pos_=="NOUN":
      return adapted_lesk(sent,x.text).name()
    else:
      return x.text
  except:
    return x.text

def wsd_text(sent):
  doc = nlp(sent)
  test_df = pd.DataFrame(doc,columns=["text"])
  test_df["wsd"] = test_df["text"].apply(one_func)
  return test_df['wsd'].str.cat(sep=' ').strip()

wsd_text(sent)

'he is sitting on the bank.n.01 of the river.n.01 watching a speedboat.n.01 .'

In [52]:
wn.synset('boat.n.01').hypernyms()
for i in wn.synset('boat.n.01').hypernym_paths()[0]:
  print(i.name())

entity.n.01
physical_entity.n.01
object.n.01
whole.n.02
artifact.n.01
instrumentality.n.03
conveyance.n.03
vehicle.n.01
craft.n.02
vessel.n.02
boat.n.01


In [53]:
df["WSD Text"] = df["Document"].map(wsd_text)

In [54]:
df.head()

,Document,Summary,WSD Text
0,japan 's nec corp. and UNK computer corp. of t...,nec UNK in computer sales tie-up,japan 's nec corp . and UNK computer corp . of...
1,the sri lankan government on wednesday announc...,sri lanka closes schools as war escalates,the sri lankan politics.n.02 on wednesday anno...
2,police arrested five anti-nuclear protesters t...,protesters target french research ship,police.n.01 arrested five anti - nuclear disse...
3,factory orders for manufactured goods rose #.#...,us september factory orders up #.# percent,factory.n.01 regulate.v.02 for manufactured we...
4,the bank of japan appealed to financial market...,bank of UNK UNK for calm in financial markets,the bank of japan appealed to financial market...


In [79]:
print(wn.synset("police.n.01").hypernym_paths()[0])
print(wn.synset("hide.n.01").hypernyms())

[Synset('entity.n.01'), Synset('abstraction.n.06'), Synset('group.n.01'), Synset('social_group.n.01'), Synset('organization.n.01'), Synset('force.n.04'), Synset('police.n.01')]
[Synset('animal_skin.n.01')]


In [22]:
a = df['WSD Text'].str.cat(sep=' ')
test_list = re.findall("[\w]+\..{1}\..{2}",a)
word_dist = FreqDist(test_list)
word_dist['abstraction.n.06']

0

In [13]:
thetha_d = 3
thetha_f = 100

def to_generalize(sent,thetha_d,thetha_f,word_dist):
  try:
    word_lst = [word for word in re.findall('[\w]+\..{1}\..{2}', sent) \
                if (word_dist[word] < thetha_f) and (len(wn.synset(word).hypernym_paths()[0]) > thetha_d)]
    for i in word_lst:
      try:
        sent = sent.replace(i, str(wn.synset(i).hypernyms()[0].name()))
      except:
        sent = sent.replace(i, str(wn.synset(i).attributes()[0].name()))
  
  except:
    return "ERROR while Generalizing"

  return sent

In [14]:
df["Generalized Text"] = df["WSD Text"].apply(lambda x: to_generalize(x,thetha_d,thetha_f,word_dist))
# xyz=23
# print(df["WSD Text"][xyz],"\n",df["Document"][xyz])

In [15]:
df[["Document","WSD Text", "Generalized Text"]]

,Document,WSD Text,Generalized Text
0,japan 's nec corp. and UNK computer corp. of t...,japan 's nec corp . and UNK computer corp . of...,japan 's nec corp . and UNK computer corp . of...
1,the sri lankan government on wednesday announc...,the sri lankan politics.n.02 on wednesday anno...,the sri lankan politics.n.02 on wednesday tell...
2,police arrested five anti-nuclear protesters t...,police.n.01 halt.v.01 five anti - nuclear diss...,force.n.04 halt.v.01 five anti - nuclear perso...
3,factory orders for manufactured goods rose #.#...,factory.n.01 regulate.v.02 for manufacture.v.0...,plant.n.01 regulate.v.02 for manufacture.v.04 ...
4,the bank of japan appealed to financial market...,the bank of japan solicitation.n.02 to financi...,the bank of japan request.n.01 to financial me...
...,...,...,...
1946,migrants should be regarded as a high-risk gro...,migrant.s.01 should be see.v.05 as a high - ri...,migrant.s.01 should be think.v.01 as a high - ...
1947,the growth rate of indonesia 's manufacturing ...,the increase.n.03 rate.v.03 of indonesia 's ma...,the process.n.06 measure.v.04 of indonesia 's ...
1948,a mozambican man suspect of murdering jorge UN...,a mozambican world.n.08 suspect.v.03 of murder...,a mozambican group.n.01 think.v.02 of murder.v...
1949,the us dollar fell to the lower ### yen level ...,the us dollar.n.04 hide.n.01 to the lower # # ...,the us dollar.n.04 animal_skin.n.01 to the low...


# **Implementation of Algorithm 1 from the paper**

Algorithm 1 Level-driven text generalization (LG)

Require: text, wsdText, F, T, θd
, θf

1: genText ← text

2: for all token ∈ text do

3: ftoken ← Frequency of token from F

4: if ftoken ≤ θf
then

5: c ← WSD of token form wsdText

6: Pc ← Path of hypernyms of c from T

7: dc ← taxonomy depth of concept c

8: fc ← Frequency of c from F

9: while fc < θf and dc > θd do

10: c ← hypernym of c from Ptoken

11: dc ← taxonomy depth of c

12: fc ← Frequency of c from F

13: end while

14: if (word of c) 6= token then

15: genText ← generalize token of genText to c

16: F ← (F \ {(token, ftoken)}) ∪ {(token, ftoken − 1)}

17: F ← (F \ {(c, fc
)}) ∪ {(c, fc + 1)}

18: end if

19: end if

20: end for

21: return genText

In [89]:
data = df[["Document","WSD Text"]]
data.columns = ["text","wsdText"]
data.head()

,text,wsdText
0,japan 's nec corp. and UNK computer corp. of t...,japan 's nec corp . and UNK computer corp . of...
1,the sri lankan government on wednesday announc...,the sri lankan politics.n.02 on wednesday anno...
2,police arrested five anti-nuclear protesters t...,police.n.01 arrested five anti - nuclear disse...
3,factory orders for manufactured goods rose #.#...,factory.n.01 regulate.v.02 for manufactured we...
4,the bank of japan appealed to financial market...,the bank of japan appealed to financial market...


In [138]:
a = data['wsdText'].str.cat(sep=' ')
b = data['text'].str.cat(sep=' ')
test_list = re.findall("[\w]+\..{1}\..{2}",a)
word_dist = FreqDist(test_list)
word_dist2 = FreqDist(b.split())
word_dist2['japan']

33

In [164]:
def algorithm_one(text,wsd_text,wsd_freq,text_freq,thetha_d,thetha_f):
  genText = text
  for token,wsd_token in zip(nlp(text),nlp(wsd_text)):
    print("Inside for", token, wsd_token)
    f_token = text_freq[token]
    if f_token <= thetha_f:
      try:
        print("Inside if",  f_token)
        c = adapted_lesk(text,wsd_token.text).name()
      except:
        c = adapted_lesk(text,wsd_token.text).hypernym_paths()[-2].name()
      print("After c", c)
      Pc = wn.synset(c).hypernym_paths()[0]
      dc = len(Pc)-1
      fc = wsd_freq[c]
      
      while fc<thetha_f and dc > thetha_d:
        try:
          print("Inside while")
          c = wn.synset(c).hypernyms()[0].name()
          dc = len(wn.synset(c).hypernym_paths()[0])-1
          fc = wsd_freq[c]
        except:
          pass

      if (wn.synset(c).lemmas()[0].name()) != token.text:
        genText = genText.replace(token.text,c)
  
  return genText


In [165]:
data["Generalized text"] = data['text'].apply( lambda x: algorithm_one(x, wsd_text(x), word_dist, word_dist2, thetha_d, thetha_f) )

Inside for japan japan
Inside if 0
After c japan.v.01
Inside for 's 's
Inside if 0


AttributeError: ignored

In [127]:
for i,j in zip(sent.split(),wsd_text(sent).split()):
  print(i,j)

he he
is is
sitting sitting
on on
the the
bank bank.n.01
of of
the the
river river.n.01
watching watching
a a
speedboat. speedboat.n.01
